# 1 数据集加载与预处理

In [ ]:
import torchvision
from torchvision import transforms

# 定义数据预处理
trans = transforms.ToTensor()

# 加载数据集（不重复下载）
mnist_train = torchvision.datasets.FashionMNIST(root="../../data", train=True, transform=trans, download=False)
mnist_test = torchvision.datasets.FashionMNIST(root="../../data", train=False, transform=trans, download=False)

## 检查数据集加载是否成功及其基本信息

In [ ]:
# 检查训练数据集和测试数据集的大小
print(f'Training dataset size: {len(mnist_train)}')
print(f'Test dataset size: {len(mnist_test)}')

# 查看一个样本的形状和标签
sample_image, sample_label = mnist_train[0]
print(f'Sample image shape: {sample_image.shape}')
print(f'Sample label: {sample_label}')

# 2 数据加载器

In [ ]:
from torch.utils.data import DataLoader

batch_size = 64

# 创建数据加载器
train_loader = DataLoader(mnist_train, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(mnist_test, batch_size=batch_size, shuffle=False)

# 3 定义模型

In [ ]:
# 定义模型参数和网络

import torch
from torch import nn  # 这种方式直接从 torch 中导入 nn 模块，使得在代码中你可以直接使用 nn 而不需要加上 torch. 前缀。使用示例：你可以直接使用 nn.Linear，而不需要 torch.nn.Linear。


num_inputs, num_outputs, num_hiddens = 784, 10, 256  # 每个图像由28*28=784个像素值组成，被分为10个类别。

# 初始化第一个全连接层的权重和偏置: 输入层->隐藏层
W1 = nn.Parameter(torch.randn(num_inputs, num_hiddens, requires_grad=True) * 0.01)  # 一个形状为(num_inputs, num_hiddens)的权重矩阵
b1 = nn.Parameter(torch.zeros(num_hiddens, requires_grad=True))
# 初始化第二个全连接层的权重和偏置: 隐藏层->输出层
W2 = nn.Parameter(torch.randn(num_hiddens, num_outputs, requires_grad=True) * 0.01)  # 一个形状为(num_hiddens, num_outputs)的权重矩阵
b2 = nn.Parameter(torch.zeros(num_outputs, requires_grad=True))

params = [W1, b1, W2, b2]  # 将所有参数收集到一个列表中，这个列表包含了网络的所有可训练参数


def relu(X):
    # ReLU激活函数
    a = torch.zeros_like(X)
    return torch.max(X, a)

def net(X):
    # 网
    X = X.reshape((-1, num_inputs))
    H = relu(X@W1 + b1)  # 这里“@”代表矩阵乘法 {输入层到隐藏层：线性变换+激活函数}
    return (H@W2 + b2)  # {隐藏层到输出层：线性变换}

# 4 定义损失函数和优化器

In [ ]:
# 损失函数
loss = nn.CrossEntropyLoss()

# 优化器
lr = 0.1
updater = torch.optim.SGD(params, lr=lr)

# 5 训练循环

In [ ]:
num_epochs = 10

for epoch in range(num_epochs):
    for X, y in train_loader:
        # 计算模型输出
        y_hat = net(X)
        # 计算损失
        l = loss(y_hat, y)
        # 梯度清零
        updater.zero_grad()
        # 反向传播
        l.backward()
        # 参数更新
        updater.step()
    
    # 每个 epoch 结束后评估模型
    with torch.no_grad():
        test_loss, correct = 0, 0
        for X, y in test_loader:
            y_hat = net(X)
            test_loss += loss(y_hat, y).sum().item()
            correct += (y_hat.argmax(1) == y).sum().item()
        print(f'Epoch {epoch + 1}, Test Loss: {test_loss / len(test_loader.dataset)}, '
              f'Accuracy: {correct / len(test_loader.dataset)}')

print("Training Complete!")